In [1]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Directorio de datos
base_dir = "/content/drive/MyDrive/Semestre 9/Visión por Computador/Semana_12/Lemon_quality/Lemon_quality"
san_dir = os.path.join(base_dir, 'good_quality')
enf_dir = os.path.join(base_dir, 'bad_quality')

In [3]:
# Listas de imágenes y etiquetas
san_images = [os.path.join(san_dir, file) for file in os.listdir(san_dir)]
enf_images = [os.path.join(enf_dir, file) for file in os.listdir(enf_dir)]
san_labels = [0] * len(san_images)
enf_labels = [1] * len(enf_images)

In [4]:
# Combinación de imágenes y etiquetas
all_images = np.array(san_images + enf_images)
all_labels = np.array(san_labels + enf_labels)

In [5]:
# División en conjuntos de entrenamiento, validación y prueba (70%-15%-15%)
X_train, X_temp, y_train, y_temp = train_test_split(all_images, all_labels, test_size=0.3, stratify=all_labels)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp)

In [6]:
# Crear carpetas de destino para las imágenes divididas
output_base = "/content/drive/MyDrive/Visión por computador/images/"
os.makedirs(output_base, exist_ok=True)
for subset in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_base, subset, 'naranjas_sanas'), exist_ok=True)
    os.makedirs(os.path.join(output_base, subset, 'naranjas_enfermas'), exist_ok=True)

In [7]:
# Función para transferir las imágenes a las carpetas específicas
def transfer_images(img_list, labels, split_name):
    for img, label in zip(img_list, labels):
        target_folder = 'naranjas_sanas' if label == 0 else 'naranjas_enfermas'
        shutil.copy(img, os.path.join(output_base, split_name, target_folder))

In [ ]:
# Transferencia de imágenes a los conjuntos de datos
transfer_images(X_train, y_train, 'train')
transfer_images(X_val, y_val, 'val')
transfer_images(X_test, y_test, 'test')

In [ ]:
# Configuración de generadores de imágenes
datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(os.path.join(output_base, 'train'), target_size=(128, 128), batch_size=32, class_mode='binary')
val_gen = datagen.flow_from_directory(os.path.join(output_base, 'val'), target_size=(128, 128), batch_size=32, class_mode='binary')
test_gen = datagen.flow_from_directory(os.path.join(output_base, 'test'), target_size=(128, 128), batch_size=32, class_mode='binary')

In [ ]:
# Crear modelo CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(learning_rate, filters):
    model = Sequential()
    model.add(Conv2D(filters, (3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters * 2, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
!pip install scikit-optimize

In [ ]:
import skopt

In [ ]:
# Optimización de hiperparámetros con Bayesian Optimization
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

# Rango de hiperparámetros
dim_learning_rate = Real(1e-6, 1e-1, prior='log-uniform', name='learning_rate')
dim_num_filters = Integer(16, 128, name='num_filters')
dimensions = [dim_learning_rate, dim_num_filters]

In [ ]:
@use_named_args(dimensions=dimensions)
def optimize_hyperparams(learning_rate, num_filters):
    temp_model = build_model(learning_rate=learning_rate, filters=num_filters)
    history = temp_model.fit(train_gen, validation_data=val_gen, epochs=10, verbose=0)
    val_accuracy = history.history['val_accuracy'][-1]
    return -val_accuracy

In [ ]:
# Realizar la optimización
result = gp_minimize(optimize_hyperparams, dimensions=dimensions, n_calls=20, random_state=42)
print("Mejores hiperparámetros encontrados:", result.x)

In [ ]:
# Crear y evaluar modelo con hiperparámetros optimizados
best_lr, best_filters = result.x
final_model = build_model(learning_rate=best_lr, filters=best_filters)
history = final_model.fit(train_gen, validation_data=val_gen, epochs=50)

In [ ]:

# Evaluación en conjunto de prueba
test_loss, test_accuracy = final_model.evaluate(test_gen)
print(f"Prueba - Pérdida: {test_loss}, Precisión: {test_accuracy}")

In [ ]:
# Métricas adicionales
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

predictions = final_model.predict(test_gen) > 0.5
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
conf_matrix = confusion_matrix(y_test, predictions)

print(f"Precisión: {accuracy}")
print(f"Exactitud: {precision}")
print(f"Sensibilidad: {recall}")
print(f"Matriz de Confusión:\n{conf_matrix}")

In [ ]:
# Visualización de la matriz de confusión
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusión')
plt.show()